In [ ]:
import numpy as np
import pandas as pd
import glob
import subprocess as sp
import os
from matplotlib import pyplot as plt
from ellipse import LsqEllipse
import cv2
import math
import scipy.stats as stats

In [ ]:
def oe_events_parser(open_ephys_csv_path, channel_names, export_path=None):
    """
    :param open_ephys_csv_path: The path to an open ephys analysis tools exported csv (using TrialReporter.ipynb)
    :param channel_names: a dictionary of the form -
                    { 1 : 'channel name' (L_eye_camera)
                      2 : 'channel name' (Arena_TTL)
                      etc..
                    }
    :param export_path: default None, if a path is specified a csv file will be saved
    :returns open_ephys_events: a pandas DataFrame object where each column has the ON events of one channel
                                and has a title from channel_names
    :returns open_ephys_off_events: same but for the OFF states (only important for the logical start-stop signal)
    """

    # Infer the active channels:
    df = pd.read_csv(open_ephys_csv_path)
    channels = df['channel'].to_numpy(copy=True)
    channels = np.unique(channels)
    df_onstate = df[df['state']==1] #cut the df to represent only rising edges
    df_offstate = df[df['state']==0] # This one is important for the ON/OFF signal of the arena
    list = []
    off_list= []
    for chan in channels: #extract a pandas series of the ON stats timestamps for each channel
        Sname = channel_names[chan]
        s = pd.Series(df_onstate['timestamp'][df_onstate['channel'] == chan], name=Sname)
        offs = pd.Series(df_offstate['timestamp'][df_offstate['channel'] == chan], name=Sname)
        list.append(s)
        off_list.append(offs)
    open_ephys_events = pd.concat(list, axis=1)
    open_ephys_off_events = pd.concat(off_list, axis=1)
    if export_path is not None :
        if not export_path in os.listdir(open_ephys_csv_path.split('events.csv')[0][:-1]):
            open_ephys_events.to_csv(export_path)
    return open_ephys_events , open_ephys_off_events

def convert_h264_mp4(path):
    files_to_convert = glob.glob(path + r'\**\*.h264', recursive=True)
    converted_files = glob.glob(path + r'\**\*.mp4', recursive=True)
    for file in files_to_convert:
        fps = file[file.find('hz') - 2:file.find('hz')]
        if len(fps) != 2:
            fps = 60
            print('could not determine fps, using 60...')
        if not str(fr'{file[:-5]}.mp4') in converted_files:
            sp.run(f'MP4Box -fps {fps} -add {file} {file[:-5]}.mp4')
            print(fr'{file} converted ')
        else:
            print(f'The file {file[:-5]}.mp4 already exists, no conversion necessary')

def validate_no_framedrop(path):
    videos_to_inspect = glob.glob(path + r'\**\*.mp4', recursive=True)
    timestamps_to_inspect = glob.glob(path + r'\**\*.csv', recursive=True)
    for vid in range(len(videos_to_inspect)):
        timestamps = pd.read_csv(timestamps_to_inspect[vid])
        num_reported = timestamps.shape[0]
        cap = cv2.VideoCapture(videos_to_inspect[vid])
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f'The video named {os.path.split(videos_to_inspect[vid])[1]} has reported {num_reported} frames '
              f'and has {length} frames, it has dropped {num_reported - length} frames')
        cap.release()

def stamp_diff_videos(path_to_stamp,stamp):
    videos_to_stamp = glob.glob(path_to_stamp + r'\**\*.mp4', recursive=True)
    for vid in videos_to_stamp:
        os.rename(vid, fr'{vid[:-4]}_{stamp}{vid[-4:]}')

def get_frame_timeseries(df,channel):
    index_range = range(0,len(df[channel][df[channel].notna()]))
    timeseries = pd.Series(df[channel][df[channel].notna()])
    timeseries = pd.Series(timeseries.values, index=index_range, name=channel)
    return timeseries

def get_closest_frame(timestamp, vid_timeseries, report_acc=None):
    """
    This function extracts a frame from a series so that it is as close as possible to a given timestamp
    :param timestamp: The time to match a frame to
    :param vid_timeseries: The time frames series to look at for a match
    :param report_acc: if set to 1, will report the accuracy of the match
    :return: index_of_lowest_diff , accuracy of match(if requested)
    """
    array = np.abs((vid_timeseries.to_numpy())-timestamp)
    index_of_lowest_diff = np.argmin(array)
    if report_acc == 1:
        accuracy = abs(vid_timeseries[index_of_lowest_diff] - timestamp)
        return index_of_lowest_diff, accuracy
    else:
        return index_of_lowest_diff

def arena_video_initial_thr(vid_path, threshold_value, show_frames=False):
    """
        This function works through an arena video to determine where the LEDs are on and when off
        :param threshold_value: value of the frame threshold
        :param show_frames: if true will show the video after thresholding
        :param  vid_path: Path to video. When ShowFrames is True a projection of the frames after threshold is presented

        :return: np.array with frame numbers and mean values after threshold
        """
    cap = cv2.VideoCapture(vid_path)
    all_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    flag = 0
    i = 0
    mean_values = []
    indexes = []
    while flag == 0:
        print('Frame number {} of {}'.format(i, all_frames), end='\r', flush=True)
        ret, frame = cap.read()
        if not ret:
            break
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        grey[grey < threshold_value] = 0
        mean_values.append(np.mean(grey))
        indexes.append(i)
        if show_frames:
            cv2.imshow('Thresholded_Frames', grey)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        i += 1
    cap.release()
    cv2.destroyAllWindows()
    frame_val = np.array((indexes, mean_values))
    return frame_val

def produce_frame_val_list(vid_paths,threshold_value):
    """
    :param vid_paths: a list of str paths to videos for analysis
    :param threshold_value: the threshold to use in order to concentrate on LEDs
    :return: frame_val_list: a list of mean pixel values for each frame after threshold
    """
    frame_val_list = []
    for vid in vid_paths:
        print(f'working on video {vid}')
        frame_val = arena_video_initial_thr(vid, threshold_value)
        frame_val_list.append(frame_val)
    print(f'done, frame_val_list contains {len(frame_val_list)} objects',flush=True)

    return frame_val_list

In [ ]:
"""
This notebook should be the main sequence for incoming experimental data - there are prerequisites for a functional process:
- the process is designed to work block-by-block
- Data will be arranged into block folders under animal folders, where each block contains the next structure:

                                       /----> arena_videos  ->[config.yaml , info.yaml] videos -> [video files, output.log] timestamps -> [csv of timestamps]

Animal_x ->date(xx_xx_xxxx) -> block_x -----> eye_videos >> LE\RE -> video folder with name -> [video.h264 , video.mp4 , params.json , timestamps.csv]

                                       \----> oe_files >> date_time(xxxx_xx_xx_xx-xx-xx) --> [events.csv] internal open ephys structure from here (NWB format only!!!)
                                                                                             /////////////////////
                                                                                         TODO: internal parsing of this file
"""

In [ ]:
animal_number = '0'
experiment_date = '24_03_2021'
block = '9'
path = rf'D:\experiments\Animal_{animal_number}\{experiment_date}\block_{block}'
print(path)
#Arena
arvid_path = path + r'\arena_videos\videos\*.mp4'
arena_videos = glob.glob(path + r'\arena_videos\videos\*.mp4')
print(arena_videos)


In [ ]:
ar_vidnames = []
for vid in arena_videos:
    vidname = vid.split(sep="\\") ; vidname = vidname[len(vidname)-1]; vidname=vidname.split(sep='.') ; vidname=vidname[0]
    ar_vidnames.append(vidname)
print(ar_vidnames)

In [ ]:
vid_path = path + r'\eye_videos'
print('converting videos...')
convert_h264_mp4(vid_path)
print('Validating videos...')
validate_no_framedrop(vid_path)

In [ ]:
print('stamping LE video')
stamp_diff_videos(vid_path+r'\LE' , 'LE')

In [ ]:
le_video = glob.glob(path + r'\eye_videos\LE\**\*.mp4')
re_video = glob.glob(path + r'\eye_videos\RE\**\*.mp4')

In [ ]:
channeldict = {
    5 : 'L_eye_TTL',
    6 : 'Arena_TTL',
    7 : 'Logical ON/OFF',
    8 : 'R_eye_TTL'
}
exp_date_time = os.listdir(fr'{path}\oe_files')[0]
oe_events , oe_off_events = oe_events_parser(path + rf'\oe_files\{exp_date_time}\events.csv', channeldict, export_path=path + rf'\oe_files\{exp_date_time}\parsed_events.csv')
ts_list = []
for chan in list(oe_events.columns):
    ts = get_frame_timeseries(oe_events,str(chan))
    ts_list.append(ts)
logical_off_series = get_frame_timeseries(oe_off_events,'Logical ON/OFF')
block_start_time = ts_list[2].values[0]
del ts_list[2]
block_end_time = logical_off_series.values[0]


In [ ]:
print(f'Arena video start: {block_start_time} \n'
      f'Arena video end: {block_end_time} \n'
      f'Block length: {block_end_time - block_start_time} Seconds')

In [ ]:
#Arena
arena_ff = ts_list[1].values[ts_list[1]>block_start_time][0]
arena_first_index = ts_list[1][ts_list[1] == arena_ff].index
arena_lf = ts_list[1].values[ts_list[1]<block_end_time][-1]
arena_last_index = ts_list[1][ts_list[1] == arena_lf].index
arena_sync_s = pd.Series(ts_list[1][arena_first_index.asi8[0] : arena_last_index.asi8[0]])

#Left eye
le_ff = ts_list[0].values[0]
le_first_index = ts_list[0][ts_list[0] == le_ff].index
le_lf = ts_list[0].values[-1]
le_last_index = ts_list[0][ts_list[0] == le_lf].index
le_sync_s = ts_list[0]

#Right eye
re_ff = ts_list[2].values[0]
re_first_index = ts_list[2][ts_list[2] == re_ff].index
re_lf = ts_list[2].values[-1]
re_last_index = ts_list[2][ts_list[2] == re_lf].index
re_sync_s = ts_list[2]

sync_time_starts = max([arena_ff, le_ff, re_ff])
sync_time_ends = min([arena_lf, le_lf, re_lf])

In [ ]:
#Define Anchor signal
anchor_signal = np.arange(sync_time_starts, sync_time_ends, 1/60)

#Create the DataFrame
sync_vids = pd.DataFrame(data=None, index=range(len(anchor_signal)), columns=['Left_eye','Arena','Arena_VideoFrame','Right_eye'])
accuracy_report = pd.DataFrame(data=None, index=range(len(anchor_signal)), columns=['Left_eye','Arena','Right_eye'])
# define dictionary for timestamp retrieval
ts_dict = {'Left_eye': ts_list[0],
           'Arena': ts_list[1],
           'Right_eye':ts_list[2]}

#Iterate over the length of the dataframe and fit
for frame in range(len(anchor_signal)):
    anchor_time = anchor_signal[frame]
    if frame % 50 == 0:
        print(f'frame {frame} out of {len(anchor_signal)}', end='\r', flush=True)
    for vid in ['Left_eye', 'Right_eye', 'Arena']:
        f,a = get_closest_frame(anchor_time, ts_dict[vid], report_acc=1)
        sync_vids[vid][frame] = f
        accuracy_report[vid][frame] = a

In [ ]:
sync_vids.Arena_VideoFrame = sync_vids.Arena - (arena_first_index.asi8[0] + 70)
sync_vids.Right_eye = sync_vids.Right_eye + 0
sync_vids

In [ ]:
sync_vids_copy = sync_vids


In [ ]:
def eye_tracking_analysis(dlc_video_analysis_csv):
    """
    :param dlc_video_analysis_csv: the csv output of a dlc analysis of one video, already read by pandas with header=1
    :param bodyparts_list: a list of bodyparts as described in the dlc csv (i.e ['Pupil_12', 'Pupil_6'....])
    :returns ellipse_df: a DataFrame of ellipses parameters (center, width, height, phi, size) for each video frame

    """

    data = dlc_video_analysis_csv
    ellipses = []
    for row in range(1, len(data)-1):
        x_values = np.array(list([float(data['Pupil_12'][row]),float(data['Pupil_1'][row]),float(data['Pupil_3'][row]),float(data['Pupil_4'][row]),float(data['Pupil_6'][row]),float(data['Pupil_8'][row]),float(data['Pupil_10'][row])]))
        y_values = np.array(list([float(data['Pupil_12.1'][row]),float(data['Pupil_1.1'][row]),float(data['Pupil_3.1'][row]),float(data['Pupil_4.1'][row]),float(data['Pupil_6.1'][row]),float(data['Pupil_8.1'][row]),float(data['Pupil_10.1'][row])]))
        X = np.c_[x_values,y_values]

        el = LsqEllipse().fit(X)
        center, width, height, phi = el.as_parameters()
        ellipses.append([center,width,height,phi])
        if row % 50 == 0:
           print(f'just finished with {row} out of {len(data)-1}', end='\r',flush=True)
    ellipse_df = pd.DataFrame(columns = ['center', 'width', 'height', 'phi'], data = ellipses)
    a = np.array(ellipse_df['height'][:])
    b = np.array(ellipse_df['width'][:])
    ellipse_size_per_frame = a*b*math.pi
    ellipse_size_per_frame
    ellipse_df['ellipse_size'] = ellipse_size_per_frame
    print('Done')
    return ellipse_df

In [ ]:
le_csv = pd.read_csv(glob.glob(path + r'\eye_videos\LE\**\*DLC*.csv')[0],header=1)

re_csv = pd.read_csv(glob.glob(path + r'\eye_videos\RE\**\*DLC*.csv')[0],header=1)


le_ellipses = eye_tracking_analysis(le_csv)
print('\n')
re_ellipses = eye_tracking_analysis(re_csv)

In [ ]:
plt.figure(figsize=[20,7])
plt.plot(anchor_signal[0:25724], le_ellipses.ellipse_size[sync_vids.Left_eye.values[0:25724]])
plt.plot(anchor_signal[0:25724], re_ellipses.ellipse_size[sync_vids.Right_eye.values[0:25724]])
_ = plt.ylim(1000,7000)


In [ ]:
arena_frame_val_list = produce_frame_val_list(arena_videos,250)

In [ ]:
l_eye_frame_val_list = produce_frame_val_list(le_video, 30)
r_eye_frame_val_list = produce_frame_val_list(re_video,30)

In [ ]:
l_eye_values = stats.zscore(l_eye_frame_val_list[0][1])
r_eye_values = stats.zscore(r_eye_frame_val_list[0][1])

In [ ]:
sync_vids.Arena_VideoFrame

In [ ]:
arena_brightness_df = pd.DataFrame(index = anchor_signal)
for ind, vid in enumerate(ar_vidnames):
    vid_val_arr = stats.zscore(arena_frame_val_list[ind][1])
    sync_list = sync_vids.Arena_VideoFrame.astype(int)
    sync_list[sync_list >= len(vid_val_arr)] = len(vid_val_arr)-1
    arena_brightness_df.insert(loc=0, column=str(vid), value=vid_val_arr[sync_list])

In [ ]:
eye_brightness_df = pd.DataFrame(index=anchor_signal)
eye_brightness_df.insert(loc=0,column='left_eye',value=l_eye_values[sync_vids.Left_eye.values.astype(int)[0:len(anchor_signal)]])
eye_brightness_df.insert(loc=0,column='right_eye',value=r_eye_values[sync_vids.Right_eye.values.astype(int)[0:len(anchor_signal)]])

In [ ]:
arena_brightness_df

In [ ]:
sec = 3600
x = 3
region = range(1020+x*sec, 1150+x*sec)

plt.figure(figsize=[20,7])
plt.plot(eye_brightness_df.iloc[region].index, eye_brightness_df.iloc[region].right_eye, label='right')
plt.plot(eye_brightness_df.iloc[region].index, eye_brightness_df.iloc[region].left_eye, label='left')
plt.plot(eye_brightness_df.iloc[region].index, arena_brightness_df.iloc[region].left_20210324T125835, label='arena1')
plt.plot(eye_brightness_df.iloc[region].index, arena_brightness_df.iloc[region].stream_20210324T125835, label='arena2')
plt.vlines(eye_brightness_df.iloc[region].index, -20, -8, linestyles={'dashed'})
plt.legend()


In [ ]:
def four_video_concat(output_name, vid1, vid2, vid3, vid4,  start_frame, shortest_vid_length, format='H264'):
    """
    :param vid1: left down
    :param vid2: right down
    :param vid3: left up
    :param vid4: right up
    :param shortest_vid_length:
    :param output_name: output file name
    :return: a concatenated video of the 4 inputs
    """
    cap0 = cv2.VideoCapture(vid1)
    cap1 = cv2.VideoCapture(vid2)
    cap2 = cv2.VideoCapture(vid3)
    cap3 = cv2.VideoCapture(vid4)
    anchor = start_frame
    fourcc = cv2.VideoWriter_fourcc(*format)
    out = cv2.VideoWriter((path + r'\\' + output_name + '.mp4'),fourcc, 60.0, (640*2,480*2))
    try:
        while cap2.isOpened():
            ar_f = sync_vids.Arena_VideoFrame[anchor]
            l_eye_f = sync_vids.Left_eye[anchor]
            r_eye_f = sync_vids.Right_eye[anchor]

            cap0.set(1,ar_f)
            ar_ret0, ar_frame0 = cap0.read()
            ar_frame0 = cv2.cvtColor(ar_frame0, cv2.COLOR_BGR2GRAY)
            ar_frame0 = cv2.resize(ar_frame0,(640,480))

            cap1.set(1,ar_f)
            ar_ret, ar_frame = cap1.read()
            ar_frame = cv2.cvtColor(ar_frame, cv2.COLOR_BGR2GRAY)
            ar_frame = cv2.resize(ar_frame,(640,480))

            cap2.set(1,l_eye_f)
            le_ret, le_f = cap2.read()
            le_f = cv2.cvtColor(le_f, cv2.COLOR_BGR2GRAY)
            le_f = cv2.flip(le_f, 0)
            le_f = cv2.resize(le_f,(640,480))

            cap3.set(1,r_eye_f)
            re_ret, re_f = cap3.read()
            re_f = cv2.cvtColor(re_f, cv2.COLOR_BGR2GRAY)
            re_f = cv2.flip(re_f, 0)
            re_f = cv2.resize(re_f,(640,480))

            eye_concat = np.hstack((le_f,re_f))
            ar_concat = np.hstack((ar_frame0, ar_frame))
            vconcat = np.vstack((eye_concat, ar_concat))

            out.write(vconcat)
            anchor += 1
            print(f'writing video frame {anchor} out of {shortest_vid_length}', end='\r', flush=True)
            if anchor > shortest_vid_length-1:
                break
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception:
        print(f'Encountered a problem with frame {anchor}, stopping concatenation')
    finally:
        cap0.release()
        cap1.release()
        cap2.release()
        cap3.release()
        out.release()
        cv2.destroyAllWindows()
        print('\n')
        print('Processed finished')

In [ ]:
four_video_concat_faster('block_9_synchronization',arena_videos[1],arena_videos[3],le_video[0],re_video[0],10,24900)


In [ ]:
import cProfile

In [ ]:
cProfile.run("four_video_concat('block_8_synchronization_profiler',arena_videos[1],arena_videos[3],le_video[0],re_video[0],15,300)")

In [ ]:

def four_video_concat_faster(output_name, vid1, vid2, vid3, vid4,  start_frame, shortest_vid_length, format='H264'):
    """
    :param vid1: left down
    :param vid2: right down
    :param vid3: left up
    :param vid4: right up
    :param shortest_vid_length:
    :param output_name: output file name
    :return: a concatenated video of the 4 inputs
    """
    cap0 = cv2.VideoCapture(vid1)
    cap1 = cv2.VideoCapture(vid2)
    cap2 = cv2.VideoCapture(vid3)
    cap3 = cv2.VideoCapture(vid4)
    anchor = start_frame
    last_ar_f = sync_vids.Arena_VideoFrame[start_frame]
    last_le = sync_vids.Left_eye[start_frame]
    last_re = sync_vids.Right_eye[start_frame]
    fourcc = cv2.VideoWriter_fourcc(*format)
    out = cv2.VideoWriter((path + r'\\' + output_name + '.mp4'),fourcc, 60.0, (640*2,480*2))
    try:
        while cap2.isOpened():
            ar_f = sync_vids.Arena_VideoFrame[anchor]
            l_eye_f = sync_vids.Left_eye[anchor]
            r_eye_f = sync_vids.Right_eye[anchor]

            if ar_f != last_ar_f + 1:
                cap0.set(1,ar_f)
            ar_ret0, ar_frame0 = cap0.read()
            ar_frame0 = cv2.cvtColor(ar_frame0, cv2.COLOR_BGR2GRAY)
            ar_frame0 = cv2.resize(ar_frame0,(640,480))
            last_ar_f = ar_f

            if ar_f != last_ar_f + 1:
                cap1.set(1,ar_f)
            ar_ret, ar_frame = cap1.read()
            ar_frame = cv2.cvtColor(ar_frame, cv2.COLOR_BGR2GRAY)
            ar_frame = cv2.resize(ar_frame,(640,480))

            if l_eye_f != last_le + 1:
                cap2.set(1,l_eye_f)
            le_ret, le_f = cap2.read()
            le_f = cv2.cvtColor(le_f, cv2.COLOR_BGR2GRAY)
            le_f = cv2.flip(le_f, 0)
            le_f = cv2.resize(le_f,(640,480))
            last_le = l_eye_f

            if r_eye_f != last_re + 1:
                cap3.set(1,r_eye_f)
            re_ret, re_f = cap3.read()
            re_f = cv2.cvtColor(re_f, cv2.COLOR_BGR2GRAY)
            re_f = cv2.flip(re_f, 0)
            re_f = cv2.resize(re_f,(640,480))
            last_re = r_eye_f

            eye_concat = np.hstack((le_f,re_f))
            ar_concat = np.hstack((ar_frame0, ar_frame))
            vconcat = np.vstack((eye_concat, ar_concat))

            out.write(vconcat)
            anchor += 1
            print(f'writing video frame {anchor} out of {shortest_vid_length}', end='\r', flush=True)
            if anchor > shortest_vid_length-1:
                break
    except Exception:
        print(f'Encountered a problem with frame {anchor}, stopping concatenation')
    finally:
        cap0.release()
        cap1.release()
        cap2.release()
        cap3.release()
        out.release()
        cv2.destroyAllWindows()
        print('\n')
        print('Processed finished')

In [ ]:
cProfile.run("four_video_concat_faster('block_10_synchronization_profiler_faster',arena_videos[1],arena_videos[3],le_video[0],re_video[0],15,3615)")